In [1]:
!pip3 install pycryptodome

Note: For the use of PyCryptoDome, I'm going to make a pull request to their repo for the name change here! For now, use Cryptodome instead of PyCrypto. 

In [14]:
from Cryptodome.PublicKey.RSA import generate, import_key

#gives the public key and private key
def write_pem(file_name,key):
    f = open(file_name, 'wb')
    f.write(key.export_key('PEM'))
    f.close()

def generate_RSA(bits=1024):
    privkey = generate(bits)
    pubkey = privkey.publickey()
    write_pem('private_key.pem',privkey)
    write_pem('public_key.pem',pubkey)


#generate_RSA()



In [19]:
from Cryptodome.Cipher import PKCS1_OAEP

def string_to_bytes(string):
    return bytes(string,'utf-8')

def encrypt_RSA(public_key_file,message):
    f = open(public_key_file,'r')
    key = import_key(f.read())
    f.close()
    cipher = PKCS1_OAEP.new(key)
    ciphertext = cipher.encrypt(string_to_bytes(open(message).read()))
    return ciphertext

#ciphertext = encrypt_RSA('public_key.pem','mydata.txt')


In [25]:
def decrypt_RSA(private_key_file, cipher):
    f = open(private_key_file,'r')
    key = import_key(f.read())
    f.close()
    ciphertxt = PKCS1_OAEP.new(key)
    message = ciphertxt.decrypt(cipher)
    return message

#decrypt_RSA('private_key.pem',ciphertext)

b'Hello, this is my data.\n\n'

In [30]:
import Cryptodome.Signature.PKCS1_PSS as pss
from Cryptodome.Hash import SHA256
#returns signature string in base64
def sign_data(private_key_file,data):
    f = open(private_key_file,'r')
    key = import_key(f.read())
    f.close()
    msg = open(data).read()
    hashed = SHA256.new(string_to_bytes(msg))
    signature = pss.new(key).sign(hashed)
    return signature

#sign_data('private_key.pem','mydata.txt')


b'[\x15hx\x7f8x|\xfe\x95\x1e\x1b\xd2\x9f\xb7\xa1\xf8\x07e\x81f\x88\x88JS)\x87#\x19xZ\x8cK\x8c\x14Y\xb2\xd6\xe8\x16\n\xf3@3\x85\'\x90Om/\xfe(;\xa3*\xca\xd8\xdc\xc0\xdd\xad\xb7\x12j\xa4H\xe6J\xf6,A\xd1\xa9\x89O\x1fu,\xd1Nq\xea^\x90jL\x0c\xeeq\xeay\x10q9\xc4\xe4\xea\x1d=^\xd3(P\xbc"%\x02\x14N\x85\xf2~!\xe6o?\xed\xc9\x88/\xb0d\xa0\xbbO\x87\xf8='

In [33]:
#returns true or false
def verify_sign(public_key_file, sign, data):
    f = open(public_key_file,'r')
    key = import_key(f.read())
    f.close()
    msg = open(data).read()
    hashed_data = SHA256.new(string_to_bytes(msg))
    verifier = pss.new(key)
    return verifier.verify(hashed_data, sign)

#signature = sign_data('private_key.pem','mydata.txt')

#print(verify_sign('public_key.pem',signature,'mydata.txt'))

    

True


In [43]:
from sq_mult import square_multiply

def enc(msg,priv,mod_p):
    return square_multiply(msg,priv,mod_p)

def dec(cipher,pub,mod_p):
    return square_multiply(cipher,pub,mod_p)

def get_key(fname, mode='r'):
    key = open(fname, mode)
    return import_key(key.read())

def int_to_bytes(integer):
     return integer.to_bytes((integer.bit_length()+7)//8,'big')

def int_from_bytes(xbytes):
    return int.from_bytes(xbytes,'big')
    

In [44]:
generate_RSA()
ciphertext = encrypt_RSA('public_key.pem','mydata.txt')
decrypt_RSA('private_key.pem',ciphertext)
signature = sign_data('private_key.pem','mydata.txt')

if verify_sign('public_key.pem',signature,'mydata.txt') == True:
    print("The signature is authentic")
else:
    print("The signature is not authentic")

i = 100
print("Encrypting: \n")
print(i)
y = encrypt_RSA('public_key.pem','somemessage.txt')
y_s = encrypt_RSA('public_key.pem','anothermessage.txt')
m = int_to_bytes(int_from_bytes(y)*int_from_bytes(y_s))
print(f"result {y}")
print(f"modified to {m}")

try:
    decrypted = decrypt_RSA('private_key.pem',m)
    print(f"Decrypted to \n {decrypted}")

except ValueError:
    print(ValueError)

The signature is authentic
Encrypting: 

100
result b'\x1c\x93\xa2\xac\\\xe9\xaf3\xd2O1\x8b\xc2\xcf\xe9\xa5\xa2\n\xd5a\xca\x94\x16\xa9\x84\xe1\xc5\xcf\xb1v\x9aA#`\x9f\xecLK\xe8GUq>\xd7xO6\xf8\xd2\xf139]SG\xefX\xc2\xb7\xe2\x15Qu\xfd\xb9L\xaf\x92\xeb\xd8\x97[[\x9a2\xb9-X\x95!\xd0mM\xa0\x8c[\x1ap\xd5\x83\xc9\x89No\xdbv@\xd5\xb8\xfbnx\xcaV\xa7\xc2\xfb\x06~_\x8b\xcc=\xa4}\x91-\x10ug\xe3\xd8\xad\xeb6\xbf\xd8l'
modified to b'\x02(\x10\\K\xed\xc5g\x07y\xbcy\x15\xc6\xae<\xd0\x00 \x1f\xb5\xfd\x94"\x9eN\xab\xa9n \x082\xa9\xe9\xcc\xdc\xf3\xda\xc4\xd2\xb8\xedFFf0O\xdf[\xdb\x0b\xc6Ym$\x95\xe0gA\xef\xd5\xc2\x91\xf1S\x8e3b\x00\x01\xe6\x8a!\xc8\xecI\xea@\xc2\xb2/D\xa3\xd2A\xe9\xcaM\xe3Q\xca\xf6+\xc8\xda\x7fS\x7f\x958\x1d\x13\xa2\xd6\xce\xdd\t\xb6u\x13\xf7\xb9v\xb7\xe3C\x8d\x9b\xb1z\xf1\xce\x9a(Il\xab\x08\x92\x99_\xa5\xb5\x9f(U\xfa\x0e\x0e\x1f\x1c\xfc%_\xda\xa3\x82\xdf_\x8cT\x1c\xa2i\x01\n\xfd\x0c\x08\x00uU\xa6k\xb1\x1b\xb4/\xfb\xc0\xb1\xd7\xdf5N\xc2"\xa4\x86\xee\xdf\xb5|{\xab^\xe3Q6\xa9#\xee\\\x91\xa2\